In [1]:
import pandas as pd
import numpy as np

pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [2]:
train_data = pd.read_parquet('../../data/initial_modelling/train_data_last_month.parquet')
test_data = pd.read_parquet('../../data/initial_modelling/test_data_last_month.parquet')

In [3]:
cat_cols = ['D_63', 'D_126', 'B_30', 'B_38']
train_data[cat_cols] = train_data[cat_cols].astype('category')
test_data[cat_cols] = test_data[cat_cols].astype('category')

In [4]:
train_data.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386351 entries, 0 to 386350
Columns: 103 entries, customer_ID to end_of_month
dtypes: category(4), datetime64[ns](2), float32(94), int64(2), string(1)
memory usage: 154.8 MB


In [5]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np

def one_hot_encode_categories(df, categorical_columns=None, drop_original=True, handle_unknown='error'):
    """
    One-hot encode categorical columns in a DataFrame.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        Input DataFrame containing categorical columns
    categorical_columns : list or None
        List of categorical column names to encode. If None, automatically detects categorical columns.
    drop_original : bool
        Whether to drop the original categorical columns
    handle_unknown : str
        Strategy for handling unknown categories in new data: 'error', 'ignore' or 'infrequent_if_exist'
    
    Returns:
    --------
    pandas.DataFrame
        DataFrame with one-hot encoded columns
    OneHotEncoder
        Fitted encoder for future transformations
    """
    # Make a copy to avoid modifying the original
    result_df = df.copy()
    
    # Automatically detect categorical columns if not specified
    if categorical_columns is None:
        categorical_columns = result_df.select_dtypes(include=['object', 'category']).columns.tolist()
    
    if not categorical_columns:
        print("No categorical columns found to encode.")
        return result_df, None
        
    # Initialize the encoder
    encoder = OneHotEncoder(sparse_output=False, handle_unknown=handle_unknown, drop='if_binary')
    
    # Fit and transform the categorical columns
    encoded_array = encoder.fit_transform(result_df[categorical_columns])
    
    # Get feature names
    feature_names = encoder.get_feature_names_out(categorical_columns)
    
    # Create a DataFrame with the encoded features
    encoded_df = pd.DataFrame(encoded_array, columns=feature_names, index=result_df.index)
    
    # Combine with the original DataFrame
    if drop_original:
        # Drop the original categorical columns
        result_df = result_df.drop(columns=categorical_columns)
    
    # Concatenate the encoded columns with the original DataFrame
    result_df = pd.concat([result_df, encoded_df], axis=1)
    
    print(f"One-hot encoded {len(categorical_columns)} categorical columns into {len(feature_names)} binary features.")
    
    return result_df, encoder

In [6]:
# Apply to your training data
train_data_oh, encoder = one_hot_encode_categories(train_data, handle_unknown='ignore')

One-hot encoded 4 categorical columns into 17 binary features.


In [7]:
train_data_oh.head()

,customer_ID,S_2,D_39,B_1,B_2,R_1,D_41,B_3,B_4,D_45,B_5,R_2,D_47,B_6,B_7,D_51,B_9,R_3,B_10,S_5,B_11,S_6,D_54,R_4,B_12,S_8,B_13,R_5,D_58,B_14,D_60,B_15,S_11,D_65,B_16,B_18,B_19,B_20,S_12,R_6,S_13,B_21,B_22,D_71,S_15,B_23,P_4,D_75,B_24,R_7,B_25,B_26,R_8,S_16,R_10,R_11,B_27,S_17,R_12,B_28,R_13,R_14,R_15,R_16,S_18,D_86,R_17,R_18,B_31,S_19,R_19,B_32,S_20,R_20,R_21,B_33,R_22,R_23,D_92,D_93,D_94,R_24,R_25,D_96,S_26,D_102,B_36,B_37,D_109,D_112,B_40,D_127,B_41,D_133,R_28,D_140,D_144,target,end_of_month,D_63_CL,D_63_CO,D_63_CR,D_63_XL,D_63_XM,D_63_XZ,B_30_0.0,B_30_1.0,B_30_2.0,B_38_1.0,B_38_2.0,B_38_3.0,B_38_4.0,B_38_5.0,B_38_6.0,B_38_7.0,D_126_1.0
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-12-04,0.001082,0.001930,1.007504,0.001772,0.004626,0.006073,0.005182,0.735001,0.189011,0.001795,0.542119,0.221899,0.001681,0.668379,0.007081,0.006369,0.431864,0.007165,0.004027,0.002629,1.009308,0.006144,0.112579,0.759416,0.074886,0.009820,0.004934,0.009725,1.008534,0.007219,0.568667,0.001157,0.000227,1.007369,0.004829,0.004406,0.085943,0.005198,0.852393,0.004330,0.008983,0.430954,0.200863,0.004899,0.007455,0.004540,0.007681,0.006707,0.003785,0.007010,0.000629,0.008428,0.003407,0.003414,0.009385,0.006325,1.008528,0.005743,0.009544,0.005875,0.002376,0.002390,0.004931,0.003480,0.004335,0.009400,1,0.007469,0.009016,0.005824,0.006831,0.008907,0.009985,1.001439,0.000014,0.006258,0.001555,0.009442,0.003827,0.002946,0.005545,0.000115,0.022741,0.906699,0.007017,0.008906,0.005981,1.005701,0.001404,1.002499,0.007171,0.003673,0.001212,0.009607,0.004753,0,2017-12-31,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,2017-12-29,0.302357,0.006711,0.819772,0.007966,0.000728,0.005235,0.001098,0.267228,0.006571,0.000905,0.399547,0.225348,0.017709,0.339235,0.006873,0.100048,0.296570,0.008128,0.000416,0.002475,1.003240,0.001278,0.029998,0.330290,0.068684,0.004056,0.000803,0.010855,0.403688,0.005928,0.449285,0.001572,0.090947,1.000319,0.001873,0.006513,0.192624,0.008571,0.001373,0.005025,0.006626,0.014687,0.300994,0.002473,0.001176,0.007739,0.005513,0.009534,0.016012,0.001688,0.009750,0.008904,0.007372,0.009307,0.006429,0.004212,1.008367,0.008966,0.003675,0.002812,0.003835,0.001464,0.002989,0.000434,0.003204,0.000289,1,0.007946,0.001519,0.005115,0.000802,0.003773,0.004639,1.009193,0.006015,0.009337,0.008599,0.008896,0.007610,0.000912,0.000542,0.008214,0.003408,0.007856,0.009352,0.005969,0.008268,1.006045,0.015016,0.002402,0.009520,0.005307,0.005935,0.009377,0.003133,0,2017-12-31,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,2017-12-12,0.007485,0.001891,0.816083,0.006484,0.007051,0.007042,0.092288,0.245794,0.004525,0.003638,0.335034,0.183796,0.079764,0.334790,0.004750,0.000374,0.296902,0.008666,0.002287,1.002712,1.008540,0.002012,0.008421,0.004028,0.007745,0.000842,0.000726,0.000684,0.002947,0.003256,0.283174,0.004786,0.002718,1.000791,0.009395,0.008664,0.193157,0.002380,0.004305,0.004392,0.008046,0.011265,0.504581,0.062451,0.004673,0.005600,0.009369,0.009639,0.003413,0.006417,0.000292,0.004796,0.001615,0.000091,0.004148,0.005096,1.005697,0.078434,0.000593,0.004610,0.002040,0.004785,0.001327,0.002912,0.007064,0.008038,1,0.005111,0.002620,0.000463,0.001379,0.006231,0.007522,1.001809,0.006854,0.003513,0.005027,0.009929,0.008960,0.009948,0.005880,0.009871,0.004860,0.002383,0.004557,0.001577,0.006629,1.002070,0.068245,0.004192,0.002416,0.007910,0.000012,0.006504,0.001522,0,2017-12-31,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,2017-12-29,0.007425,0.015824,0.962010,0.004479,0.004799,0.009391,0.015474,0.079261,0.072228,0.007786,0.404808,0.176701,0.027479,0.001530,0.028851,0.007758,0.192981,0.058831,0.010140,0.005843,1.007432,0.001348,0.072455,0.323510,0.056811,0.004176,0.081144,0.020386,0.553116,0.008326,0.363006,0.005494,0.090868,0.544162,0.001465

In [8]:
encoder.feature_names_in_

array(['D_63', 'B_30', 'B_38', 'D_126'], dtype=object)

In [9]:
encoder.transform(test_data[encoder.feature_names_in_])

array([[0., 0., 1., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 1.],
       ...,
       [0., 1., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 1.]])

In [10]:
test_data_oh = pd.concat([test_data.drop(columns=encoder.feature_names_in_),pd.DataFrame(encoder.transform(test_data[encoder.feature_names_in_]), columns=encoder.get_feature_names_out(),index=test_data.index)], axis=1)

In [11]:
test_data_oh.shape

(405607, 116)

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_val = train_test_split(train_data_oh, test_size=0.3, random_state=0, stratify = train_data_oh['target'])

In [13]:
y_train = X_train['target']
y_val = X_val['target']

In [14]:
X_train.shape, X_val.shape

((270445, 116), (115906, 116))

In [15]:
X_train[['customer_ID', 'target']].to_parquet('../../data/initial_modelling/y_train.parquet', index = False)
X_val[['customer_ID', 'target']].to_parquet('../../data/initial_modelling/y_val.parquet', index = False)

In [16]:
drop_cols = ['customer_ID', 'target', 'S_2', 'end_of_month']

In [17]:
X_train = X_train.drop(columns = drop_cols)
X_val = X_val.drop(columns = drop_cols)

In [18]:
X_val.head(10)

,D_39,B_1,B_2,R_1,D_41,B_3,B_4,D_45,B_5,R_2,D_47,B_6,B_7,D_51,B_9,R_3,B_10,S_5,B_11,S_6,D_54,R_4,B_12,S_8,B_13,R_5,D_58,B_14,D_60,B_15,S_11,D_65,B_16,B_18,B_19,B_20,S_12,R_6,S_13,B_21,B_22,D_71,S_15,B_23,P_4,D_75,B_24,R_7,B_25,B_26,R_8,S_16,R_10,R_11,B_27,S_17,R_12,B_28,R_13,R_14,R_15,R_16,S_18,D_86,R_17,R_18,B_31,S_19,R_19,B_32,S_20,R_20,R_21,B_33,R_22,R_23,D_92,D_93,D_94,R_24,R_25,D_96,S_26,D_102,B_36,B_37,D_109,D_112,B_40,D_127,B_41,D_133,R_28,D_140,D_144,D_63_CL,D_63_CO,D_63_CR,D_63_XL,D_63_XM,D_63_XZ,B_30_0.0,B_30_1.0,B_30_2.0,B_38_1.0,B_38_2.0,B_38_3.0,B_38_4.0,B_38_5.0,B_38_6.0,B_38_7.0,D_126_1.0
18987,0.002942,0.032364,1.009652,0.030380,0.003394,0.003644,0.030278,1.103598,0.170297,0.001116,0.870468,0.198136,0.032464,0.339421,0.002276,0.005110,0.286894,0.058036,0.018683,0.000184,1.009922,0.004170,0.139378,0.753548,0.182328,0.009547,0.003163,0.082787,1.002155,0.096714,0.402411,0.000850,0.007406,1.003898,0.005725,0.006780,0.232633,0.001491,0.683061,0.006359,0.002923,0.101965,0.201421,0.025244,0.005060,0.004411,0.006587,0.001149,0.000354,0.005937,0.004396,0.000977,0.006220,0.007481,0.002995,0.340270,1.000757,0.044601,0.009202,0.005135,0.008831,0.004389,0.002665,0.007052,0.000411,0.003030,1,0.002724,0.006581,0.006841,0.003352,0.001604,0.006436,1.008488,0.000250,0.001507,1.003092,0.003671,0.000721,0.001944,0.006663,0.009705,0.059349,0.009400,0.007266,0.037875,0.007491,1.003468,0.003596,0.004512,0.009682,0.001885,0.001829,0.006862,0.002297,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
148768,0.003151,0.289360,0.112041,0.506558,0.248659,0.219512,0.285715,0.035022,0.015426,1.003749,0.313216,0.010247,0.497624,0.008243,0.847865,0.203579,0.027216,0.051729,0.241159,0.007127,1.008456,0.003981,0.012643,0.409083,0.024011,0.509825,0.829960,0.070060,0.119509,0.002624,0.447048,0.006833,0.925555,0.096094,0.448615,0.652263,0.186399,0.003189,0.290402,0.009004,0.006974,0.009133,0.304376,0.475709,0.009399,0.668169,0.002788,0.005868,0.112694,0.009988,1.000419,0.004540,1.002322,0.001649,0.004278,0.003349,1.004932,0.259825,0.005332,0.004730,0.008157,0.505702,0.005779,0.004850,0.004145,0.000412,1,0.001876,0.008532,0.003677,0.000692,0.007162,0.000507,0.003705,0.004623,0.001002,0.000466,0.005219,0.004939,0.007461,0.005173,0.001373,0.009986,0.320224,0.006207,0.295553,0.003759,0.002430,0.447166,0.000224,0.003176,0.882811,0.003107,0.005478,0.000888,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
5095,0.121072,0.373771,0.073985,0.002255,0.509808,0.553177,0.057456,0.337240,0.010931,1.003704,0.228855,0.011451,0.537469,0.007773,0.613306,0.205537,0.029276,0.018375,0.292809,0.008503,1.003255,0.000687,0.005793,0.009102,0.002595,0.003536,0.161302,0.047139,0.664607,0.004816,0.000815,0.000647,1.003179,0.215519,0.709570,1.004510,0.190505,0.007498,0.003836,0.000253,0.005911,0.008949,0.801601,0.402477,0.000141,0.136083,0.000701,0.008710,0.082875,0.009615,0.000991,0.007100,0.007073,0.006075,0.009140,0.007365,1.006962,0.033155,0.008815,0.007093,0.006679,0.003492,0.001747,0.003660,0.007915,0.003268,1,0.002242,0.007015,0.001018,0.007165,1.007730,0.007276,0.006190,0.006953,0.009629,0.008240,0.001959,0.006258,0.004572,0.000499,0.003579,0.004298,0.064751,0.006173,0.373623,0.004195,0.009410,0.379247,0.007006,0.007949,0.006319,0.007265,0.009457,0.006679,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
143200,0.210048,0.038128,1.005942,0.001762,0.002745,0.005699,0.053901,0.780319,0.067780,0.004955,0.621924,0.129850,0.044468,0.007813,0.006940,0.006647,0.191286,0.031222,0.024544,0.006213,1.006558,0.006776,0.070140,0.318015,0.060706,0.009979,0.087315,0.046872,0.756810,0.060184,0.405121,0.006676,0.087281,0.688696,0.000524,0.003460,0.354833,0.009048,0.513206,0.005232,0.004304,0.050005,0.408997,0.026427,0.009670,0.069179,0.009099,0.006850,0.004513,0.002772,0.004220,0.006567,0.002968,0.005717,0.003777,0.003909,1.007556,0.034514,0.007033,0.002382,0.009694,0.009191,0.009616,0.005307,0.007465,0.009357,1,0.008350,0.001105,0.

In [19]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

In [20]:
X_train_scaled = pd.DataFrame(X_train_scaled, columns = X_train.columns)
X_val_scaled = pd.DataFrame(X_val_scaled, columns = X_train.columns)

In [21]:
X_train_scaled.head(10)

,D_39,B_1,B_2,R_1,D_41,B_3,B_4,D_45,B_5,R_2,D_47,B_6,B_7,D_51,B_9,R_3,B_10,S_5,B_11,S_6,D_54,R_4,B_12,S_8,B_13,R_5,D_58,B_14,D_60,B_15,S_11,D_65,B_16,B_18,B_19,B_20,S_12,R_6,S_13,B_21,B_22,D_71,S_15,B_23,P_4,D_75,B_24,R_7,B_25,B_26,R_8,S_16,R_10,R_11,B_27,S_17,R_12,B_28,R_13,R_14,R_15,R_16,S_18,D_86,R_17,R_18,B_31,S_19,R_19,B_32,S_20,R_20,R_21,B_33,R_22,R_23,D_92,D_93,D_94,R_24,R_25,D_96,S_26,D_102,B_36,B_37,D_109,D_112,B_40,D_127,B_41,D_133,R_28,D_140,D_144,D_63_CL,D_63_CO,D_63_CR,D_63_XL,D_63_XM,D_63_XZ,B_30_0.0,B_30_1.0,B_30_2.0,B_38_1.0,B_38_2.0,B_38_3.0,B_38_4.0,B_38_5.0,B_38_6.0,B_38_7.0,D_126_1.0
0,0.720652,-0.066572,0.208630,0.609512,-0.297008,-0.011088,-0.048303,0.421480,12.728937,-0.811324,0.406045,0.682814,-0.173667,0.991528,0.663489,0.879433,0.703248,5.488661,0.044387,-0.745245,0.972543,-0.038405,12.181761,0.342773,8.445914,-0.137736,-0.271297,3.257563,1.061424,0.302090,-1.013085,0.512715,-0.109683,0.448496,0.000606,-0.008712,2.575421,0.863342,0.971101,0.047153,-0.884099,-0.280690,-0.305813,-0.140703,0.595118,-0.250798,0.170017,0.478800,5.512335,-0.144987,0.914214,0.929508,0.033029,0.005267,0.262921,7.792575,0.203855,0.571548,-0.466969,-0.767180,-0.124025,-0.153897,195.865814,-0.493427,-0.778512,-0.070329,0.0,-0.981706,0.666305,-0.085526,-0.486353,-0.945399,0.181710,0.000882,-0.812984,0.329872,182.653893,0.721508,0.570621,-0.275717,0.175758,-0.183385,14.227012,0.219090,0.098792,-0.002888,-0.510610,-0.196261,-0.176032,-0.111306,0.878038,0.397168,0.285756,-0.081433,126.522951,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.114763,6.311282,-0.868704,-0.274477,0.002373,4.573184,1.211365,-0.375803,0.175997,0.392321,-0.051711,-0.407804,4.422931,-0.012218,1.864739,0.963020,-0.209778,0.093657,7.323852,0.143947,-0.195173,-0.000829,-0.162234,-0.645492,-0.143320,0.806630,0.578047,1.612274,0.058673,-0.874756,-0.177837,0.826345,1.095350,-0.680250,6.864859,2.107103,0.022628,-0.925491,-0.006595,0.629871,79.380005,-0.364673,0.361428,4.683537,-0.081127,0.483319,-0.586058,0.136846,2.797631,0.106176,0.574635,0.420349,-0.207775,0.420807,-0.711521,0.322657,0.009362,1.384695,-0.176249,-0.974585,0.363429,0.249572,0.211891,0.172510,0.815717,0.754786,0.0,0.007273,-0.413702,-0.226694,0.642903,0.779045,-0.371885,-0.994087,-0.638108,-0.465351,0.486470,0.212762,-0.718463,0.744261,0.169079,-0.621975,0.649527,1.064372,0.036858,6.504683,-0.753308,-0.452205,3.567435,0.484824,0.513855,-0.761116,-0.717117,-0.352621,0.146600,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.119952,3.845824,-0.852362,-0.269085,-0.231215,1.862500,0.423915,-0.406826,0.283578,0.470427,0.082769,-0.168257,0.573230,-0.014297,1.583444,0.918891,-0.323294,-0.045983,4.505279,-0.363152,-0.781930,0.791859,-0.088516,-0.654071,0.146309,-0.188001,-0.075812,1.558769,1.005683,-0.267648,-0.211872,0.762531,1.089267,-0.542960,5.559654,2.092349,-0.113952,0.227411,-0.008532,0.141297,79.106053,-0.318398,0.363639,0.539701,0.329551,0.250327,0.358466,0.702048,2.838806,-0.404307,0.479017,0.079448,0.699103,0.615336,-0.841506,-0.757869,-0.229058,0.098496,0.810862,-0.972668,-0.071460,0.007562,-0.872689,0.341181,-0.287673,0.889453,0.0,0.939718,0.590563,0.626145,0.532242,0.194203,-0.633079,-0.999124,-0.090383,0.556708,0.098949,-0.463139,0.170315,-0.117296,-0.341138,-0.626460,13.314737,2.053042,-0.236166,3.986624,0.012079,-161.701094,-0.204281,-0.017967,-0.457354,-0.625285,0.621110,-0.811063,-0.767246,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.724787,-0.143107,0.000901,-0.160489,-0.990091,-0.014991,-0.288450,0.366301,1.041096,0.640631,-0.127575,0.557697,-0.216758,0.980493,-0.042234,0.891939,1.260962,-0.081387,-0.118830,0.225218,0.720373,-0.542138,2.310355,1.211590,3.574158,0.003165,-0.032224,-0.048534,0.051086,2.582279,1.403307,0.083289,-0.103797,0.061231,0.016168,-0.013344,14.353135,0.816301,1.459997,0.537053,-0.748534,8.210548,-0.998701,-0.186355,-0.838739,-0.004224,-0.034208,0.143216,0.029701,-0.211931,0.308911,0.025549,-

In [22]:
X_val_scaled.head(10)

,D_39,B_1,B_2,R_1,D_41,B_3,B_4,D_45,B_5,R_2,D_47,B_6,B_7,D_51,B_9,R_3,B_10,S_5,B_11,S_6,D_54,R_4,B_12,S_8,B_13,R_5,D_58,B_14,D_60,B_15,S_11,D_65,B_16,B_18,B_19,B_20,S_12,R_6,S_13,B_21,B_22,D_71,S_15,B_23,P_4,D_75,B_24,R_7,B_25,B_26,R_8,S_16,R_10,R_11,B_27,S_17,R_12,B_28,R_13,R_14,R_15,R_16,S_18,D_86,R_17,R_18,B_31,S_19,R_19,B_32,S_20,R_20,R_21,B_33,R_22,R_23,D_92,D_93,D_94,R_24,R_25,D_96,S_26,D_102,B_36,B_37,D_109,D_112,B_40,D_127,B_41,D_133,R_28,D_140,D_144,D_63_CL,D_63_CO,D_63_CR,D_63_XL,D_63_XM,D_63_XZ,B_30_0.0,B_30_1.0,B_30_2.0,B_38_1.0,B_38_2.0,B_38_3.0,B_38_4.0,B_38_5.0,B_38_6.0,B_38_7.0,D_126_1.0
0,-0.027043,-0.008303,0.214722,3.999342,-0.415648,-0.032777,-0.236887,2.840724,3.197316,-0.784018,1.493226,0.685270,-0.162820,0.996512,-0.059614,-0.034351,0.649234,0.612988,-0.026307,-0.969406,0.985733,-0.194200,1.975463,0.879301,1.916334,0.844422,-0.273809,0.481736,1.149074,14.472981,0.391334,-0.837397,-0.102355,0.446615,-0.009540,-0.002866,0.946436,-0.716430,1.587599,0.247813,-0.531699,3.459983,-0.670290,-0.136600,-0.034558,-0.264685,0.270760,-0.780762,-0.172318,0.133179,-0.139759,-0.807825,0.179678,0.379877,-0.397900,61.978401,-0.793193,-0.214190,0.817842,0.007280,0.741860,-0.192219,-0.479757,0.361834,-0.916141,-0.391004,0.0,-0.454694,0.291674,0.335841,-0.339278,-0.685906,0.266932,0.006788,-0.950895,-0.699377,182.248177,-0.270250,-0.859973,-0.619104,0.329807,0.885723,5.737749,0.001331,0.442207,0.037380,0.496170,-0.080955,-0.219991,-0.199566,0.891882,-0.652427,-0.637892,0.343819,-0.584490,0.0,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.026152,1.977417,-0.772095,82.579435,41.892905,1.096275,0.873062,-0.502559,-0.010512,190.166273,-0.242722,-0.393526,1.652610,0.002134,2.423644,1.871312,-0.319089,0.524382,2.026683,0.107400,0.698155,-0.230775,-0.122152,0.176148,-0.048577,97.506007,1.909570,0.361359,-0.177215,-0.595869,0.613941,0.302992,0.994659,-0.687369,3.038118,1.355212,-0.110460,-0.390125,0.661469,0.768509,0.112053,-0.148823,-0.324361,1.699715,0.790407,2.254807,-0.460920,0.136186,0.795826,0.905963,195.043822,-0.108815,189.119264,-0.695431,-0.141378,-0.383458,0.014365,0.970107,0.054843,-0.072427,0.608900,92.549985,0.127307,-0.064258,-0.176054,-0.914162,0.0,-0.624397,0.673589,-0.280677,-0.867580,0.408288,-0.901395,-0.998408,-0.079261,-0.800620,-0.915206,0.035950,-0.030048,0.470277,0.032579,-0.736246,0.346679,0.920340,0.231659,2.084902,-0.250509,-165.483360,1.646153,-0.957756,-0.380007,160.534905,-0.382360,0.072491,-0.840385,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.476587,2.629630,-0.813934,-0.641837,86.941473,2.841442,-0.118794,0.442998,-0.103613,190.157691,-0.505524,-0.386610,1.808119,0.000722,1.734810,1.890117,-0.311408,0.055798,2.503295,0.320756,-0.322263,-0.869041,-0.235538,-0.640330,-0.314378,-0.316959,0.143797,0.144557,0.641865,-0.244856,-1.611444,-0.876161,1.087405,-0.538188,4.833828,2.096330,-0.016612,0.438119,-0.014428,-0.954103,-0.056911,-0.155965,1.346306,1.401186,-0.969470,0.235117,-0.862971,0.688319,0.538846,0.834905,-0.807092,0.393422,0.341454,0.120713,0.831324,0.359891,0.407134,-0.277173,0.741553,0.393028,0.317078,-0.358253,-0.658687,-0.294562,0.571232,-0.343435,0.0,-0.551123,0.376513,-0.798910,0.418156,197.366734,0.432375,-0.995922,0.385206,0.927804,0.505021,-0.608735,0.229518,-0.100204,-0.900240,-0.306803,-0.274475,0.164984,0.224787,2.705248,-0.163289,-164.330033,1.360412,0.241541,0.553115,0.158898,0.448680,0.852597,0.211339,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.855924,0.036233,0.210643,-0.723294,-0.527623,-0.022028,-0.134239,1.829273,1.073892,-0.052971,0.718962,0.293196,-0.115970,0.000844,-0.045920,-0.019589,0.292719,0.236289,0.027784,-0.034289,0.325771,0.310740,0.829484,-0.009748,0.406847,0.927942,-0.051585,0.142035,0.780411,8.622656,0.404850,0.273173,-0.006920,0.052880,-0.045327,-0.009851,3.739888,0.735903,1.186977,0.026007,-0.312287,1.440071,0.027164,-0.131779,0.841770,-0.018837,0.754565,0.326920,-0.136477,-0.470421,-0

In [23]:
from xgboost import XGBClassifier

In [24]:
model = XGBClassifier(random_state=0)

In [25]:
model.fit(X_train_scaled, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=0, ...)

In [26]:
y_pred = model.predict(X_val_scaled)

In [27]:
from sklearn.metrics import classification_report
print(classification_report(y_val, y_pred, digits = 4))

              precision    recall  f1-score   support

           0     0.9204    0.9286    0.9245     89020
           1     0.7566    0.7342    0.7452     26886

    accuracy                         0.8836    115906
   macro avg     0.8385    0.8314    0.8349    115906
weighted avg     0.8824    0.8836    0.8829    115906



In [28]:
# Test set results

In [29]:
test_data_oh.drop(columns = drop_cols, inplace = True)

In [30]:
test_X_scaled = scaler.transform(test_data_oh)
test_X_scaled = pd.DataFrame(test_X_scaled, columns = test_data_oh.columns)

In [31]:
y_test = test_data['target']

In [32]:
y_pred_test = model.predict(test_X_scaled)

In [33]:
print(classification_report(y_test, y_pred_test, digits = 4))

              precision    recall  f1-score   support

           0     0.9370    0.9274    0.9322    307381
           1     0.7799    0.8048    0.7921     98226

    accuracy                         0.8977    405607
   macro avg     0.8584    0.8661    0.8621    405607
weighted avg     0.8989    0.8977    0.8982    405607

